Questo notebook associa ad ogni tiro presente nel pbp la posizione di tiro in base ai poligoni definiti dal notebook create_poligons

In [2]:
from shapely.geometry import Point, Polygon
import pandas as pd

shot_map_name = 'shot_map'

df = pd.read_csv('../csvs_notebooks/polygons.csv')
df_shot_map = pd.read_csv('../csvs_notebooks/' + shot_map_name + '.csv')

In [3]:
# retrieve dictionary with polygons
polygons = {}

polygon_ids = set(df.shapeId)
for polygon_id in polygon_ids:
    df_temp = df[df.shapeId == polygon_id]
    points = list(zip(df_temp.pointX, df_temp.pointY))
    polygons.update({polygon_id:points})

In [4]:
def retrieve_polygon_id(row):
    """
    function to retrieve the polygon of the point
    """
    p = Point(row['xx'], row['yy'])
    for key, value in polygons.items():
        poly = Polygon(value)
        if poly.contains(p) or poly.touches(p):
            return key
    return -1

In [5]:
df_shot_map['sector_id'] = df_shot_map.apply(lambda row: retrieve_polygon_id(row), axis=1)

In [6]:
df_shot_map.groupby(['sector_id'])['sector_id'].count()

sector_id
0    415
1      9
2     61
3     83
4     33
5     22
6    134
7    131
8     30
Name: sector_id, dtype: int64

In [8]:
df_shot_map['sector_id'] = df_shot_map['sector_id'].astype(int)
df_shot_map.to_csv('../csvs_notebooks/' + shot_map_name + '_with_sector_id.csv')